In [4]:
# ThoroughBet Simulation


## Load necessary modules

In [30]:
import numpy as np

from utils import timestamp, YEAR
from utils.arrayview import ArrayView, TimeseriesView

from prediction.models.fit_model import TSModel
from prediction.models.model_parameters import ModelParameters
from prediction.models.factor_management import FactorList
from prediction.models.preprocessing import load_slices

from prediction.models.prediction import factornames_trimmed
from prediction.models.parameters import factor_build_end

## Load data

In [2]:
av = ArrayView.from_file('../datadev/brain_final2cut.av.bcolz')

In [32]:
tsav = load_slices(path='../datadev/')

## Preprocessing

In [4]:
pars = ModelParameters(av, oos_start=factor_build_end+YEAR, depth=3, lmbd=10, verbose=True)

In [5]:
fl = FactorList(av, factornames_trimmed)

In [6]:
fl.preprocess(pars)
factors = fl.asmatrix()

INFO:models:Getting factors from av and rescaling...


. . . . .

INFO:models:Filling in missing values...
INFO:models:Computing each factor as linear combination of all the others...


 . . . . .

INFO:models:Number of missing patterns: 7754


 . . . . . . .


## Fit conditional logit model

In [7]:
tsmod = TSModel(factors, tsav, pars)

In [8]:
tsmod.fit_slices()

INFO:models:Fitting slice 0
INFO:models:Fitting slice 1
INFO:models:Fitting slice 2
INFO:models:Fitting slice 3
INFO:models:Fitting slice 4
INFO:models:Fitting slice 5
INFO:models:Fitting slice 6
INFO:models:Fitting slice 7
INFO:models:Fitting slice 8
INFO:models:Fitting slice 9


In [9]:
tsmod.write_simdata('simdata.p')